In [2]:

from google.colab import files
uploaded = files.upload()

Saving test.data to test.data
Saving test.label to test.label
Saving train.data to train.data
Saving train.label to train.label


In [4]:
import csv 
import math
from collections import defaultdict

print('starting to train..')

label_path = 'train.label'
data_path = 'train.data'
number_of_class=20;
number_of_words=61188

# construct label dict
f = open(label_path)
reader = csv.reader(f,delimiter=" ")
label={}
i=0;
for rows in reader:
    label[i]=int(rows[0])-1
    i+=1
    
num_of_doc=len(label)

# Nk - number of doc per class
Nk=[0]*number_of_class;
for i in label:
    Nk[label[i]]=Nk[label[i]]+1;


# nk[c][w]= number of docs with word 'w' in class 'c'
min_count=5
nk = [[min_count] * number_of_words for i in range(number_of_class)]   
f = open(data_path)
reader = csv.reader(f,delimiter=" ")
for rows in reader:
    nk[label[int(rows[0])-1]][int(rows[1])-1]+=1


PrWgivenC = [[0.0] * number_of_words for i in range(number_of_class)]  
for c in range(0,number_of_class):
    for w in range(0,number_of_words):
        PrWgivenC[c][w]=math.log(nk[c][w])-math.log(Nk[c])+10
#         PrWgivenC[c][w]=nk[c][w]/Nk[c]
        
PrC=[0]*number_of_class
for c in range(0,number_of_class):
    PrC[c]=math.log(Nk[c])-math.log(num_of_doc)
#     PrC[c]=Nk[c]/num_of_doc

def getClass(wc):
    global PrC,PrWgivenC
    PrCgivenW=[0.0]*number_of_words
    sum_pr=0.0;
    for c in range(0,number_of_class):
#         pr=1.0;
        pr=0;
        for w in range(0,number_of_words):
            pr=pr+(wc[w]*PrWgivenC[c][w]) + (1-wc[w])*(1-PrWgivenC[c][w])
#             pr=pr*(wc[w]*PrWgivenC[c][w] + (1-wc[w])*(1-PrWgivenC[c][w]))
#         PrCgivenW[c]=pr*PrC[c]
        PrCgivenW[c]=pr+PrC[c]
#         print("class: "+str(c)+"  pr: "+str(PrCgivenW[c]))
        sum_pr+=PrCgivenW[c]
    if sum_pr==0:
        return -1
    
    max_c=0
    for c in range(0,number_of_class):
        PrCgivenW[c]=PrCgivenW[c]/sum_pr
        if PrCgivenW[max_c]<PrCgivenW[c]:
            max_c=c
    return max_c;




# wc=[0,0,0,1,0,0,1,1]
# wc=[0,1,1,1,0,0,1,1] 
# wc=[1,0,1,0,1,1,0,0] 

# print(getClass(wc))
    

print("done!")


starting to train..
done!


In [5]:
print('starting to buid wc for test')
test_label_path = 'test.label'
test_data_path = 'test.data'
f = open(test_label_path)
reader = csv.reader(f,delimiter=" ")
tlabel={}
i=0;
for rows in reader:
    tlabel[i]=int(rows[0])-1
    i+=1

no_of_test_doc=7505

wc = [[0] * number_of_words for i in range(no_of_test_doc)] 

f = open(test_data_path)
reader = csv.reader(f,delimiter=" ")
for rows in reader:
    wc[int(rows[0])-1][int(rows[1])-1]=1

print("created wc")

starting to buid wc for test
created wc


In [6]:
print("starting to test")
f = open(test_label_path)
reader = csv.reader(f,delimiter=" ")
tlabel={}
i=0;
for rows in reader:
    tlabel[i]=int(rows[0])-1
    i+=1
    
correct=0
incorrect=0
for i in range(0,7505):
    l=getClass(wc[i])
    if l==tlabel[i]:
        correct+=1
    else:
        incorrect+=1
#     print("actual class:"+str(label[i])+"  pred:"+str(l))

print("final result: c: "+str(correct)+"  e: "+str(incorrect))

starting to test
final result: c: 251  e: 7254


In [0]:
print("accuracy: "+str(correct/7505))

0.597601598934044